In [27]:
# initialize modules and globals
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import os
import shutil
driver = webdriver.Chrome()

In [28]:
filetype = 'proficiency'
local_html = './data/ma/' + filetype + '/sample next gen html/ma_sample.htm'
yearID = 'ctl00_ContentPlaceHolder1_ddYear'
gradeID = 'ctl00_ContentPlaceHolder1_ddGrade'
groupID = 'ctl00_ContentPlaceHolder1_ddSubGroup'
exportName = 'ctl00_ContentPlaceHolder1_spExport'
destination = '/Users/esegr1/Documents/repos/tcf-benchmarking-data/data/ma/' + filetype + '/nextgen_mcas/'

In [29]:
# load html to data
with open(local_html) as html:
    soup = BeautifulSoup(html, 'html.parser')

# scrape available years, grades, and student groups
years = []

for item in soup.find(id=yearID).children:
    if item.string != '\n':
        years.append(item.string)

print(years)

grades = []

for item in soup.find(id=gradeID).children:
    if item.string != '\n':
        grades.append(item.string)
        
print(grades)        

groups = []

for item in soup.find(id=groupID).children:
    if item.string != '\n':
        groups.append(item.string)

groups = ['All Students', 'Econ. Disadvantaged', 'Afr. Amer./Black', 'Hispanic/Latino', 'White']
print(groups)

['2019', '2018', '2017']
['ALL (03-08)', '03', '04', '05', '06', '07', '08', '10']
['All Students', 'Econ. Disadvantaged', 'Afr. Amer./Black', 'Hispanic/Latino', 'White']


In [30]:
# loop through all combinations of year/grade/group, downloading each file and renaming it
# recreate html page, with each option selected
for year in years:
    for grade in grades:
        for group in groups:
            for item in soup.find(id=yearID).children:
                try:
                    if item.string == year:
                        item['selected'] = 'selected'
                    if item.string != year:
                        del item['selected']
                except TypeError:
                    pass
            for item in soup.find(id=gradeID).children:
                try:
                    if item.string == grade:
                        item['selected'] = 'selected'
                    if item.string != grade:
                        del item['selected']
                except TypeError:
                    pass
            for item in soup.find(id=groupID).children:
                try:
                    if item.string == group:
                        item['selected'] = 'selected'
                    if item.string != group:
                        del item['selected']
                except TypeError:
                    pass
            html_file = open('./data/ma/' + filetype + '/sample next gen html/generated/ma_html_temp.html','w')
            html_file.write(soup.prettify())
            html_file.close()
            driver.get('file:///'+os.getcwd()+'/' + '/data/ma/' + filetype + '/sample next gen html/generated/ma_html_temp.html')

            # simulate export click with selenium
            download_button = driver.find_element_by_id(exportName)
            download_button.click()

            # wait for file to download
            time.sleep(5)

            # rename file
            filename = 'mcas_nextgen_' + grade.replace('.', '').replace('/', '') + '_' + group.replace('.', '').replace('/', '') + '_' + year.replace('.', '').replace('/', '') + '.xlsx'
            os.rename('/Users/esegr1/Downloads/NextGenMCAS.xlsx','/Users/esegr1/Downloads/' + filename)
            shutil.move(os.path.join('/Users/esegr1/Downloads/',filename), os.path.join(destination, filename))